Importing the Libraries

In [1]:
import time
import pyspark.sql.functions as f

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *

from pyspark.sql.functions import udf
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import desc

from itertools import islice

Reading the file from HDFS

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark Assignment") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-04 20:09:05,895 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Py4JError: An error occurred while calling None.org.apache.spark.sql.SparkSession. Trace:
py4j.Py4JException: Constructor org.apache.spark.sql.SparkSession([class org.apache.spark.SparkContext, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:750)



In [ ]:
sc = spark.sparkContext
rdd = sc.textFile('wrk/akhgabzhalilov/clickstream.csv')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-04 20:07:59,766 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2023-10-04 20:08:02,386 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


Removing header from the file and parsing it

In [6]:
rdd = rdd.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it 
)
rdd = rdd.map(lambda l: l.split('\t'))
rdd.take(5)

NameError: name 'rdd' is not defined

Schema for the data

In [ ]:
rdd = rdd.map(lambda p: Row(session_id=int(str(p[1]) + str(p[0])), event_type=str(p[2]), event_page=str(p[3]), timestamp=int(p[4])))
rdd.take(5)

Creating DataFrame

In [ ]:
df = spark.createDataFrame(rdd)
df.createOrReplaceTempView('clickstream')
df.cache()
df.show(5)
df.count()

Removing Events

In [ ]:
df = df[df['event_type'] != 'event']
df.show(5)
df.count()

In [ ]:
df = df.orderBy(['session_id', 'timestamp'], ascending=True)
df.show(15)
df.count()

In [ ]:
df.groupby('session_id').count().count()

In [ ]:
df.groupby('session_id').count().orderBy('count', ascending=False).show(10)

In [ ]:
df[df['session_id'] == 1367635].count()

Calculating the result

In [ ]:
dataCollect = df.collect()
len(dataCollect)

In [ ]:
result = {}

In [ ]:
for row in dataCollect:
    session_id = row['session_id']
    event_type = row['event_type']
    event_page = row['event_page']
    
    if session_id not in result:
        result[session_id] = []
        
    if event_type == 'page':
        result[session_id].append(event_page)
    else:
        result[session_id].append('error')

In [ ]:
len(result)

In [ ]:
rows = []

In [ ]:
for k, v in result.items():
    # create route
    route = []
    for page in v:
        if page == 'error': break
        if len(route) > 0:
            if route[-1] == page:
                continue
        route.append(page)
        
    route = '-'.join(route)
        
    # Add session_id, route to DataFrame
    rows.append(Row(k, route))

In [ ]:
len(rows)

In [ ]:
df2 = spark.createDataFrame(rows, ['session_id', 'route'])
df2.show(5, False)

In [ ]:
df2[df2['session_id'] == 507].show()

Proof-Check

In [ ]:
df[df['session_id'] == 514].show(50, False)

In [ ]:
df3 = df2.groupby('route').count().orderBy('count', ascending=False).limit(30)

In [ ]:
df3.show(100, truncate=False)

In [ ]:
for row in df3.collect():
    print(row)
    break

In [ ]:
print(df3.limit(30).toPandas().to_string(index=False, header=False))

In [ ]:
!hadoop fs -ls

In [ ]:
!hadoop fs -get sga_output.csv /home/akhgabzhalilov/